## 8 컴퓨터 비전을 위한 딥러닝

1 합성곱 신경망 소개 <br>
2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기 <br>
3 사전 훈련된 모델 활용하기 <br>
4 요약 <br><br>

컴퓨터 비전은 2011~2015년 사이의 초기 딥러닝의 부흥을 이끈 영역 <br>
합성곱 신경망(Convolutional Neural Network)라고 부르는 딥러닝 모델의 종류가 그 당시 이미지 분류 대회에서 좋은 결과를 얻었음 <br><br>


### 8.1 합성곱 신경망 소개
여기서는 컨브넷 정의와 컨브넷이 컴퓨터 비전 관련 작업에 잘 맞는 이유에 대한 이론적배경 <br>
2장에서 밀집 연결신경망으로 풀었던 정확도는 97.8%, MNIST 숫자 이미지 분류에 컨브넷을 사용할 예정이고 성능이 다를 예정

#### 코드 8-1 간단한 컨브넷 만들기

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

컨브넷이 배치 차원을 제외하고 (image_height, image_width, image_channles) 크기의 입력 텐서를 사용한다는 점이 중요하다, <br>
이 예제에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력 텐서를 처리하도록 설정해야 한다. 

#### 코드 8-2 모델의 summary() 메서드 출력

In [2]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

- Conv2D와 MaxPooling2D층의 출력은 (height, width, channels) 크기의 rank-3 tensor <br>
- 높이와 너비 차원은 모델이 깊어질수록 작아지는 경향이 있다. <br>
- 채널의 수는 Conv2D층에 전달된 첫번쨰 매개변수에 의해 조절됨 (32개, 64개, 128개), <br>
- 마지막 Conv2D 층의 출력크기는 (3,3, 128)이다. 즉, 128개의 채널을 가진 3*3 크기의 특성 맵(feature map)이다. <br>
- 그 다음 단계는 이 출력을 밀집 연결 분류기로 주입한다. 이 분류기는 1D 벡터를 처리하는데 이전 층이 rank-3 tensor라서 Dense층 이전에 Flatten 층으로 먼저 3D출력을 1D 텐서로 펼쳐야한다<br>
- 마지막으로 10개의 클래스를 분류하기 위해 마지막층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용한다 <br><br>


위에 만들어놓은 모델로 MNIST를 훈련하겠다, 2장의 MNIST 예제 코드를 많이 재사용할 것임 <br>
소프트 맥스 활성화 함수의 출력을 바탕으로 10개의 클래스를 분류하기 떄문에 범주형 크로스엔트로피 손실을 사용할것임 <br>
레이블이 정수이므로 희소한 크로스엔트로피 손실인 sparse_categorical_crossentropy를 사용할 것임 <br>
#### 코드 8-3 MNIST 이미지에서 컨브넷 훈련하기

In [3]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32")/255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32")/255
model.compile(optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 42s 4ms/step - loss: 0.1497 - accuracy: 0.9538
Epoch 2/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0435 - accuracy: 0.9867
Epoch 3/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0303 - accuracy: 0.9907
Epoch 4/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0231 - accuracy: 0.9931
Epoch 5/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0192 - accuracy: 0.9940


테스트 데이터에서 모델을 평가한다. 

#### 코드 8-4 컨브넷 평가하기

In [4]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 정확도: {test_acc:.3f}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0226 - accuracy: 0.9925
테스트 정확도: 0.993


2장의 완전 연결네트워크는 97.8%의 테스트 정확도를 얻은 반면 기본적인 컨브넷은 99.1%의 테스트 정확도를 얻었다. 왜그럴까?


#### 8.1.1 합성곱 연산
Dene층은 입력 특성 공간에 있는 전역 패턴(ex, MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습한다,
이미지일 경우 작은 2D 윈도우로 입력에서 패턴을 찾는다. 앞의 예에서 이 윈도우는 모두 3*3의 크기였음 <br>

- 학습된 패턴은 평행 이동 불변성(translation invariant)을 가짐 <br>: 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른곳에서도 이 패턴을 인식할 수 있다. Fully Connected Layer에서는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야한다. 이런 성질은 컨브넷이 이미지를 효율적으로 처리하게 만들어준다(근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식되지 않는다). 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있다. <b>근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식되지 않는다</b> <br>
- 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있음 <br>: 첫 번째 합성곱 층이 edge같은 작은 지역 패턴을 학습한다. 2번째 합성곱 층은 첫번쨰 층의 특성으로 구성된 더  성된 더 큰 패턴을 학습하는 식. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있음, <b>근본적으로 우리가 보는 세상은 공간적 계층 구조를 가지고 있기 때문</b> <br>
    - 합성곱 연산은 특성 맵(feature map)이라고 부르는 rank-3 tensor에 적용 <br>
    - 2개의 공간 축(높이와 너비)과 깊이 축(채널축이라고도 부름) <br>
    - RGB 이미지는 3개의 컬러 채널(빨간색, 녹색, 파란색)을 가지므로 깊이 축의 차원이 3이 됨 <br>
    - MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 1(gray scale)임
    - 합성곱 연산은 입력 특성맵에서 작은 패치(patch)들을 추출하고 이런 모든 패치에 같은 변환을 적용하여 축력 특성 맵(output feature map)을 만듦 <br>
    - 출력 텐서의 깊이는 층의 매개변수로 결정되기 때문에 상황에 따라 다름, 이렇게 되면 깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하지 않는다. 그 대신 일종의 filter를 의미함. 필터는 입력 데이터의 어떤 특성을 인코딩, ex) 고수준으로 보면 하나의 필터가 '입력에 얼굴이 있는지'를 인토딩 할 수 있다. <br>
    - MNIST 예제에서는 첫번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32) 크기의 특성 맵을 출력함. <br>
    - 즉, 입력에 대해 32개의 필터를 적용함, 32개의 출력 채널 각각은 26*26 크기의 배열 값을 가짐, 이 값은 입력에 대한 필터의 응답맵 (response map)이다. 입력의 각 위치에서 필터 패턴에 대한 응답을 나타냄. <br>
    - 특성 맵이란 말이 의미하는 것은 다음과 같다: 각 차원은 하나의 특성( 또는 필터)이고 rank-2 tensor인 output[:, :, n]은 입력에 대한 이 필터 응답을 나타내는 2d 공간상의 map이다. <br>
    - 핵심적인 2개의 파라미터로 정의 됨 <br>:
        - <b> 입력으로부터 뽑아낼 패치의 크기</b>: 전형적으로 3*3 or 5*5 크기를 사용 <br>
        - <b> 특성 맵의 출력 깊이</b>: 합성곱으로 계산한 필터 개수, 여기서는 깊이 32로 시작해서 깊이 128로 끝남 <br>

연산과정
케라스의 Conv2D층에서 이 파라미터는 Conv2D(ouput_depth, (window_height, window_width))처럼 첫번째와 두번쨰 매개변수로 전달 <br>
↓ <br>
3D 입력 특성 맵 위를 3*3 or 5*5 크기의 윈도우가 sliding 하면서 모든 위치에서 3D 특성 패치 ((window_height, window_width, input_depth) 크기)를 추출하는 방식으로 합성곱이 작동 <br>
: 이런 3D 패치는 합성곱 커널(convolution kernel)이라고 불리는 하나의 하습된 가중채 행렬과의 텐서곱셈을 통해 (output_depth,) 크기의 1D 벡터로 변환 <br>
↓ <br>
동일한 커널이 모든 패치에 걸쳐서 재사용 <br>
↓ <br>
변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성 <br>
↓ <br>
출력 특성 맵의 공간산 위치는 입력 특성 맵의 같은 위치에 대응됨 <br>
↓ <br>
3*3 윈도우를 사용하면 3D 패치 input[i-1:i+2, j-1:j+2, :] 로부터 벡터 output[i, j, :]가 만들어짐 <br><br>

** 여기서 2가지 이유로 출력 높이와 너비는 입력의 높이, 너비와 다를 수 있음 **
- 경계 문제: 입력 특성 맵에 패딩을 추가하여 대응 할 수 있음 <br>
- stride의 사용 여부에 따라 다름 <br>


#### 경계 문제와 패딩 이해하기
- 5*5 크기의 특성 맵을 생각 해볼때, 3*3 크기의 총 9개의 격자를 형성하니, 크기가 조금 줄어든다. <br>
- 입력과 동일한 높이와 너비를 가진 출력 특성 맵을 얻고 싶다면 패딩(padding)을 사용할 수 있음 <br>
- Conv2D층에서 패딩은 padding 매개변수로 설정 할 수 있고 2개의 값이 가능함<br>
- "valid"는 패딩을 사용하지 않는다는 뜻, "same"은 "입력과 동일한 높이와 너비를 가진 출력을 만들기 위해 패딩한다"라는 뜻. padding 매개변수의 기본값은 "valid"이다. <br>

#### 합성곱 스트라이드 이해하기 
출력 크기에 영향을 미치는 다른 요소는 스트라이드이다. 두 번의 연속적인 위도우 사이의 거리가 stride라고 불리는 합성곱의 파라미터이다, 기본값은 1이다. 스트라이드가 1보다 큰 스트라이드 합성곱도 가능함. <br>
- ex) stride=2라는 뜻은 특성 맵의 너비와 높이가 2의 배수로 downsampling 되었다는 뜻, 스트라이드 합성곱은 분류 모델에서 드물게 사용된다. <br>
- 분류 모델에서는 특성 맵을 다운샘플링하기 위해 stride 대신 max pooling연산을 사용하는 경우가 많다 <br>


#### 8.1.2 최대 풀링 연산
- 최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최댓값을 출력 <br>
- 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형변환(합성곱 커널을 적용하는 대신 하드코딩된 최댓값 추출 연산을 사용함 <br>

#### 코드 8-5 최대 풀링 층이 빠진 잘못된 구조의 컨브넷

In [5]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation='softmax')(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

model_no_max_pool.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 128)       73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 61952)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                619530    
Total params: 712,202
Trainable params: 712,202
Non-trainable params: 0
_____________________________________________________

위 설정에는 2가지 문제가 있음

- 특성의 공간적 계층 구조를 학습하는 데 도움이 되지 않는다, 3번째 층의 3*3 윈도우는 초기 입력의 7*7 윈도우 영역에 대한 정보만 담고 있다, 3번째 층의 3*3 윈도우는 초기 입력의 7*7 윈도우 영역에 대한 정보만 담고 있다. 마지막 합성곱 층의 특성이 전체 입력에 대하 정보를 가지고 있어야 한다. <br>
- 작은 모델치고는 너무 많은 가중치고, 심각한 과대적합이 발생할 것 <br>

downsampling을 하는 이유는 처리할 특성 맵의 가중치 개수를 줄이기 위해서이다. 또한, 연속적인 합성곱 층이 (원본 입력에서 커버되는 영역 측면에서) 점점 커진 윈도우를 통해 바라보도록 만들어진 필터의 공간적인 계층 구조를 구성한다. <br><br>

*** 그 외 방법들 ***
1) 합성곱 층에서 스트라이드를 사용할 수 있음 <br>
2) 입력 패치의 채널별 평균값을 계산항 변환하는 평균 풀링(average pooling) <br>
    - 최대 풀링이 다른 방법들보다 더 잘 작동하는 편, 그 이유는 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩하는 경향이 있기 때문, that's why it is called 특성의 지도 (map) <br>
3) Best (가장 납득할 만한 서브샘플링 전략은 먼저 (Stride가 없는 합성굡으로) 조밀한 특성 맵을 만들고 그 다음 작은 패치에 대해 최대로 활성화된 특성을 고르는 것 <br><br>


### 8.2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기
- 데이터 증식(data augmentation)방법 <br>
- 사전 훈련된 네트워크로 특성을 추출하는 것(97.5%의 정확도를 얻게 됨)과 사전 훈련된 네트워크를 세밀하게 튜닝하는 것(최종 모델은 09.5% 정확도를 얻을 것) <br>
=> 위 3가지 전략은 반드시 알고 있어야함 <br><br>


#### 8.2.1 작은 데이터셋 문제에서 딥러닝의 타당성
- 모델을 훈련하기에 '충분한 샘플'이라는 것은 상대적이다. <br>
- speech-to-text 모델을 조금만 변경해서 완전히 다른 문제에 재사용할 수 있음 <-- 이것이 바로 특성 재사용 <br>
    - 특히, 컴퓨터 비전에서는 (보통 ImageNet 데이터셋에서 훈련된) 사전 훈련된 모델들이 내려받ㅇ르 수 있도록 많이 공개되어 있어 매우 적ㅇ느 데이터에서 강력한 비전 모델을 만드는데 사용할 수 있다. <br><br>
    
    
#### 8.2.2 데이터 내려받기
원본 데이터셋을 https://www.kaggle.com/c/dogs-vs-cats/data에서 내려받을 수 있음

In [6]:
import os 

os.getcwd()

# ..은 상위폴더
# .은 현재폴더
# ~는 절대경로

'/aiffel/aiffel/workplace/Aiffel_Quest/DeepLearningwithPython_2ndEdition'

In [5]:
pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 9.4 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110700 sha256=3d3699a0254fe8afbcf86e7c5407e37df5933c6b4cc4999954a8e0f41bf4d90a
  Stored in directory: /aiffel/.cache/pip/wheels/d2/ed/a5/da3a0cfb13373d1ace41cafa4f2467d858c55c52473ba72799
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [21]:
# 확인용으로 하나 만듦
# !mkdir ../kaggle  

# 교과서에 나오는데로 코드필사, 하지만 내 디렉토리 환경에 맞게
# !cp kaggle.json ../kaggle
# 교과서에 나오는데로 필사, 이거 안해주면 디렉토리 에러난다
!cp kaggle.json ~/.kaggle/

# 만들어주고 
!chmod 600 ../kaggle/kaggle.json

# 복사해주고
!cat ../kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
{"username":"shinnew9918","key":"2f14477510c1c5bce1ef10b2d843b54e"}

In [16]:
!kaggle competitions download -c dogs-vs-cats 

/bin/bash: kaggle: command not found


In [35]:
# 내 디렉토리에 맞게 복사해줌
!cp dogs-vs-cats.zip ../kaggle

#### 코드 8-6 이미지를 훈련, 검증, 테스트 디렉토리로 복사하기

In [3]:
import os, shutil, pathlib

original_dir = pathlib.Path("../kaggle/dogs-vs-cats/train")
new_base_dir = pathlib.Path("../kaggle/dogs-vs-cats/cats_vs_dogs_small")
def make_subset(subset_name, start_index, end_index):
    # start_index에서 end_index까지의 고양이와 강아지 이미지를 new_base/dir{subset_name}/cat(또는/dog)으로 복사하기 위한 
    # 유틸리티 함수'subset_name'은 'train', 'validation', 'test' 중 하나 
    for category in ("cat", "dog"):
        dir = new_base_dir/subset_name/category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir/fname,
                           dst=dir/fname)
            
make_subset("train", start_index=0, end_index=1000)  # 카테고리마다 처음 1000개의 이미지를 훈련 서브셋으로 만듦
make_subset("validation", start_index=1000, end_index=1500)  # 카테고리마다 그 다음 500개의 이미지를 검증 서브셋으로 만듦
make_subset("test", start_index=1500, end_index=2500)  # 카테고리마다 그 다음 1000개의 이미지를 테스트 서브셋으로 만듦

#### 8.2.3 모델 만들기

첫 번째 에제에서 봤던 일반적ㅇ니 모델 구조를 동일하게 재사용, Conv2D(relu 활성화 함수사용) MaxPooling2D층을 번갈아 쌓은 컨브넷이다. 이전보다 이미지가 크고 복잡한 문제이기 때문에 모델을 좀 더 크게 만들겠다. Conv2D와 MaxPooling2D단계를 하나 더 추가.이렇게 하면 모델의 용량을 늘리고 Flatten 층의 크기가 너무 커지지 않도록 특성 맵의 크기를 줄일 수 있다. <br>

__cf) Note__ <br>
- 특성 맵의 깊이는 모델에서 점진적으로 증가하지만 (32에서 256까지), 특성 맵의 크기는 감소함 (180*180에서 7*7까지). 이는 거의 모든 컨브넷에서 볼 수 있는 전형적인 패턴임 <br>
- 이진 분류 문제이므로 모델은 하나의 유닛(크기가 1인 Dense층)과 sigmoid활성화 함수로 끝난다. 이 유닛은 모델이 보고 있는 샘플이 한 클래스에 속할 확률을 인코딩할 것 <br>
- 마지막 작은 차이점 하나는 Rescaling층으로 모델이 시작되는 것. 이 층은 (원래 [0, 255]범위의 값인 이미지 입력을 [0, 1] 범위로 스케일 변환이다. <br>

#### 코드 8-7 강아지 vs 고양이 분류를 위한 소규모 컨브넷 만들기

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

TypeError: Inputs to a layer should be tensors. Got: <keras.layers.core.Flatten object at 0x7f6b200a5ee0>

#### 코드 8-8 모델 훈련 설정하기

In [ ]:
model.compile(loss="binary_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])

#### 8.2.4 데이터 전처리

데이터는 네트워크에 주입되기 전에 부동 소수점 타읍의 텐서로 적절하게 전처리되어 있어야 한다. 네트워크에 주입하려면 대략 다음 과정을 따라. <br>
1. 사진 파일을 읽음 <br>
2. JPEG 콘텐츠를 RGB 픽셀 값으로 디코딩함 <br>
3. 그 다음 부동 소수점 타입의 텐서로 변환 <br>
4. 동일한 크기의 이미지로 바꾼다 (여기에서는 180*180을 사용한다)
5. 배치로 묶는다(하나의 배치는 32개의 이미지로 구성)

image_dataset_from_directory() 함수를 사용하면 디스크에 있는 이미지 파일을 자동으로 전처리된 텐서의 배치로 변환하는 데이터 파이프라인을 빠르게 구성할 수 있다. <br>

#### 코드 8-9 image_dataset_from_directory를 사용하여 이미지 읽기

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
                                            new_base_dir/"train",
                                            image_size=(180, 180),
                                            batch_size=32)
validation_dataset = image_dataset_from_directory(
                                            new_base_dir/"validation",
                                            image_size=(180, 180),
                                            batch_size=32)
test_dataset = image_dataset_from_directory(
                                            new_base_dir/"test",
                                            image_size=(180, 180),
                                            batch_size=32)

__cf) 텐서플로 Dataset 객체 이해하기__

텐서플로는 머신러닝 모델을 위한 효율적인 입력 파이프라인을 만들 수 있는 tf.data.API를 제공함. <br>
Dataset 객체는 반복자(iterator)임, for루프에 사용할 수 있으며 일반적으로 입력 데이터와 레이블의 배치를 반환. Dataset 객체를 바로 케라스 모델의 fit() 메서드에 전달할 수 있음 <br>

Dataset클래스는 데이터셋을 조작하기 위한 함수형 스타일의 API도 제공.

In [ ]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)  
# from_tensor_slices() 클래스 메서드를 사용하여 하나의 넘파이 배열 또는 넘파이 배열의 튜플이나 딕셔너리에 Dataset을 만들 수 있음

In [ ]:
# 처음에는 이 데이터셋이 하나의 샘플을 반환
for i, element in enumerate(dataset):
    print(element.shape)
    if i>=2:
        break

In [ ]:
# .batch() 메서드를 사용하면 데이터의 배치가 반환됨
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i>=2:
        break

일반적으로 다음과 같은 유용한 메서드를 사용할 수 있음. <br>

- shuffle(buffer_size): 버퍼안의 원소를 섞는다 <br>
- prefetch(buffer_size): 장치 활용도를 높이기 위해 GPU 메모리에 로드할 데이터를 미리 준비한다 <br>
- map(callable): 임의의 변환을 데이터셋의 각 원소에 적용함(callable 함수는 데이터셋이 반환하는 1개의 원소를 입력으로 기대)

특히 .map() 메서드는 자주 사용함, 예를 들어 예제 데이터셋의 원소 크기를 (16,)에서 (4,4)로 변환

In [ ]:
reshaped_dataset = dataset.map(lambda x:tf.reshape(x(4,4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i>=2:
        break

#### 코드 8-10 Dataset이 반환하는 데이터와 레이블 크기 확인하기

In [ ]:
for data_batch, labels_batch in train_dataset:
    print("데이터 배치 크기:", data_batch.shape)
    print("레이블 배치 크기:", labels_batch.shape)
    break

#### 코드 8-11 Dataset을 사용하여 모델 훈련하기

In [ ]:
callbacks =[
    keras.callbacks.ModelCheckpoint(
        filepath = "convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_dataset = validation_dataset,
    callbacks = callbacks)

#### 코드 8-12 훈련 과정의 정확도와 손실 그래프 그리기

In [ ]:
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy)+1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epothcs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()